In [1]:
import requests

import pandas as pd
import numpy as np

import camelot
import json

from glob import glob

from datetime import datetime as dt

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 200

In [3]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10]

## 1. OSC with CLUNI (Clave Única de Inscripción)

In [4]:
print(str(dt.now())[:19], '- Loading OSCs with CLUNI (Unique Id)\n')

# check if file already exist in folder
sirfosc_files = glob('../data/SIRFOSC/csv/*.csv')

if len(sirfosc_files) > 0: 
    most_recent_file = max([pd.to_datetime(dt[-23:-4], format='%Y-%m-%d-%H-%M-%S') for dt in sirfosc_files])
    print(str(dt.now())[:19], f'- File already exists. Retrieving most recent ({str(most_recent_file)})...\n')
    file_name = '../data/SIRFOSC/csv/report-rfosc-' + str(most_recent_file).replace(' ', '-').replace(':', '-') + '.csv'
    
    df_osc_cluni = pd.read_csv(file_name, low_memory=False)
    df_osc_cluni = df_osc_cluni.drop(df_osc_cluni.columns[0], axis=1)
    print(str(dt.now())[:19], '- Data loaded in dataframe')
    
# if not, download from gov site
else:
    print(str(dt.now())[:19], '- Info is not locally available. Data will be downloaded from SII portal')
    with open("../params/sirfosc_query_filters.json", "r") as  params:
        jParams = json.load(params)

    CLUNI_URL = f"http://www.sii.gob.mx/portal/organizaciones/excel/?cluni={jParams['CLUNI']}&nombre={jParams['NOMBRE']}&acronimo={jParams['ACRONIMO']}&rfc={jParams['RFC']}&status_osc={jParams['STATUS_OSC']}&status_sancion={jParams['STATUS_SANCION']}&figura_juridica={jParams['FIGURA_JURIDICA']}&estado={jParams['ESTADO']}&municipio={jParams['MUNICIPIO']}&asentamiento={jParams['ASENTAMIENTO']}&cp={jParams['CP']}&rep_nombre={jParams['REP_NOMBRE']}&rep_apaterno={jParams['REP_APATERNO']}&rep_amaterno={jParams['REP_AMATERNO']}&num_notaria={jParams['NUM_NOTARIA']}&objeto_social={jParams['OBJETO_SOCIAL']}&red={jParams['RED']}&advanced=1"

    # todo: do i really need all this data to talk to the server?
    payload={}
    headers = {
      'Upgrade-Insecure-Requests': '1',
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
    }
    
    print(str(dt.now())[:19], '- Requesting data to server...')
    cluni_response = requests.request("GET", CLUNI_URL, headers=headers, data=payload)

    CLUNI_NOW = str(dt.now())[:19].replace(' ', '-').replace(':', '-')

    print(str(dt.now())[:19], '- Applying some cleaning processing')
    with open(f"../data/SIRFOSC/txt/report-rfosc-{CLUNI_NOW}.txt", "w+") as f:
        f.write(cluni_response.text)
        
    print(str(dt.now())[:19], '- Saving retreived data as text file')
    df_osc_cluni = pd.DataFrame([row.split('","') for row in cluni_response.text.split('\n')])
    df_osc_cluni[0] = df_osc_cluni[0].str.replace('"', '')
    df_osc_cluni[df_osc_cluni.shape[1]-1] = df_osc_cluni[df_osc_cluni.shape[1]-1].str.replace('"\r', '')
    df_osc_cluni = df_osc_cluni.rename(columns=df_osc_cluni.iloc[0]).iloc[1:]
    df_osc_cluni = df_osc_cluni[:-1].reset_index(drop=True)
    
    print(str(dt.now())[:19], '- Writing request as csv in SIRFOSC folder\n')
    df_osc_cluni.to_csv(f"../data/SIRFOSC/csv/report-rfosc-{CLUNI_NOW}.csv")
    
    print(str(dt.now())[:19], '- Data loaded in dataframe')

2021-03-03 07:04:20 - Loading OSCs with CLUNI (Unique Id)

2021-03-03 07:04:20 - File already exists. Retrieving most recent (2021-02-22 07:58:12)...

2021-03-03 07:04:21 - Data loaded in dataframe


In [5]:
with_active_cluni = (df_osc_cluni.ESTATUS == 'ACTIVA')
with_valid_status = (df_osc_cluni['ESTATUS DE LA REPRESENTACION'] == 'VIGENTE')

# df_osc_cluni = df_osc_cluni[with_active_cluni & with_valid_status]
df_osc_cluni = df_osc_cluni[df_osc_cluni.RFC.str.len().isin([12, 13])]
df_osc_cluni = df_osc_cluni.reset_index(drop=True)
df_osc_cluni = df_osc_cluni[['RFC', 'CLUNI', 'RAZON SOCIAL', 'ESTATUS', 'ESTATUS DE LA REPRESENTACION', 'FIGURA JURIDICA', 'FECHA DE CONSTITUCION']]

In [6]:
new_col_names = [(c, c.lower().replace(' ', '_')) for c in df_osc_cluni.columns]

In [7]:
df_osc_cluni = df_osc_cluni.rename(columns=dict(new_col_names))

In [8]:
df_osc_cluni = df_osc_cluni.rename(columns={'razon_social': 'cluni_razon_social', 'estatus_de_la_representacion':'cluni_representacion'})

In [9]:
df_osc_cluni['fecha_de_constitucion'] = pd.to_datetime(df_osc_cluni.fecha_de_constitucion)

In [10]:
df_osc_cluni[['estatus', 'cluni_representacion']] = df_osc_cluni[['estatus', 'cluni_representacion']].apply(lambda col: col.str.lower())

In [11]:
df_osc_cluni.loc[:, 'cluni_year'] = '2020'

In [12]:
df_osc_cluni.head(5)

,rfc,cluni,cluni_razon_social,estatus,cluni_representacion,figura_juridica,fecha_de_constitucion,cluni_year
0,VCO980914457,VCO98091405018,Voluntades por Coahuila,activa,vencida,Asociación Civil,1998-09-14,2020
1,UEF0303179P9,UEF0303170101D,Unión Estatal Femenil FEPRA,activa,vigente,Asociación Civil,2003-03-17,2020
2,FOJ041002PB3,FOJ04100219013,Federación de Organizaciones Juveniles,inactiva,vigente,Asociación Civil,2004-10-02,2020
3,ARP950406IF8,ARP95040609019,Aguilas Reales Pro Dignidad del Discapacitado,activa,vencida,Asociación Civil,1995-04-06,2020
4,FIM971201896,FIM97120122015,Formación Integral de la Mujer,inactiva,vigente,Institución de Asistencia Privada,1997-09-18,2020


## 2. Donatarias Autorizadas (Directorio SAT)

In [13]:
print(str(dt.now())[:19], '- Loading OSCs from Directorio Donatarias Autorizadas (SAT)\n')

# check if file already exist in folder
ddonaut_files = glob('../data/DonatariasAutorizadas/*.xls')

if len(ddonaut_files) > 0: 
    most_recent_file = max([pd.to_datetime(dt[-23:-4], format='%Y-%m-%d-%H-%M-%S') for dt in ddonaut_files])
    print(str(dt.now())[:19], f'- File already exists. Retrieving most recent ({str(most_recent_file)})...\n')
    file_name = '../data/DonatariasAutorizadas/report-sat-2020-' + str(most_recent_file).replace(' ', '-').replace(':', '-') + '.xls'
    
    df_osc_donaut_sat = pd.read_excel(file_name, skiprows = range(0, 27), usecols = "A:O")
    df_osc_donaut_sat.columns = df_osc_donaut_sat.iloc[0]
    df_osc_donaut_sat = df_osc_donaut_sat.loc[1:].reset_index(drop=True)
    print(str(dt.now())[:19], '- Data loaded in dataframe')

else:
    
    print(str(dt.now())[:19], '- Info is not locally available. Data will be downloaded from SAT portal')
    # todo: create params to download directorio donaut
    YEAR = '2020'
    DONAUT_NOW = str(dt.now())[:19].replace(' ', '-').replace(':', '-')
    SAT_URL = f'http://omawww.sat.gob.mx/documentossat/Documents/DirectorioDonatariasAutorizadas{YEAR}.xls'

    print(str(dt.now())[:19], '- Requesting data to server...')
    response = requests.get(SAT_URL)

    print(str(dt.now())[:19], '- Saving retreived data as XLS file\n')
    with open(f"../data/DonatariasAutorizadas/report-sat-{YEAR}-{DONAUT_NOW}.xls", "wb") as f:
        f.write(response.content)
        
    df_osc_donaut_sat = pd.read_excel(
        f"../data/DonatariasAutorizadas/report-sat-{YEAR}-{DONAUT_NOW}.xls"
        , skiprows = range(0, 27)
        , usecols = "A:O"
        )

    df_osc_donaut_sat = df_osc_donaut_sat.rename(columns=df_osc_donaut_sat.iloc[0]).iloc[1:].reset_index(drop=True)
    print(str(dt.now())[:19], '- Data loaded in dataframe')

2021-03-03 07:04:50 - Loading OSCs from Directorio Donatarias Autorizadas (SAT)

2021-03-03 07:04:50 - Info is not locally available. Data will be downloaded from SAT portal
2021-03-03 07:04:50 - Requesting data to server...
2021-03-03 07:06:19 - Saving retreived data as XLS file



FileNotFoundError: [Errno 2] No such file or directory: './data/DonatariasAutorizadas/report-sat-2020-2021-03-03-07-04-50.xls'

In [ ]:
df_osc_donaut_sat = (
    df_osc_donaut_sat
    .drop_duplicates('RFC')
    [['RFC', 'FECHA DE OFICIO']]
    .rename(columns={
        'RFC': 'rfc'
        , 'FECHA DE OFICIO': 'fecha_oficio_sat'
    })
)

In [ ]:
df_osc_donaut_sat = df_osc_donaut_sat[df_osc_donaut_sat.rfc.str.len().isin([12, 13])].reset_index(drop=True)

In [ ]:
df_osc_donaut_sat['fecha_oficio_sat'] = pd.to_datetime(df_osc_donaut_sat.fecha_oficio_sat)

In [ ]:
df_osc_donaut_sat.loc[:, 'sat_year'] = '2020'

In [ ]:
df_osc_donaut_sat.head(5)

## 3. Donatarias Autorizadas (Resolución Miscelánea Fiscal)

In [ ]:
with open("../params/dof_page_breaks.json", "r") as  params:
    jParams = json.load(params)

In [ ]:
DOF_YEARS = list(jParams.keys())

In [ ]:
def fix_multiple_cols_name(df):
    ncols = df.shape[1]
    if ncols == 2: return(df)
    elif ncols > 2:
        # names can generate multiple columns due to bad parsing
        name_cols = list(range(1, ncols))
        df[1] = df[name_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
        df = df[[0, 1]]
        return(df)
    else:
        # print('Check dataframe')
        # print(df.head(3))
        return(pd.DataFrame(columns=[0, 1]))

In [ ]:
df_osc_donaut_dof = pd.DataFrame(columns=['RFC', 'razon_social', 'dof_year'])

for year in DOF_YEARS:
    
    if year != '2020': continue
    print(str(dt.now())[:19], f'- Getting OSC data from "Diario Oficial de la Federacion ({year})"\n')
    DOF_YEAR = year
    
    print(str(dt.now())[:19], "- Create dataframe to store OSC's data")
    df_donaut_rms = pd.DataFrame(columns=[0, 1])
    
    # parsing first page where osc start (truncated)
    first = jParams[DOF_YEAR]['parse']['start']['pages']
    print(str(dt.now())[:19], f"- Parsing first page ({first}) where OSC's data begins")
    start_tl = jParams[DOF_YEAR]['parse']['start']['top-left']
    start_br = jParams[DOF_YEAR]['parse']['start']['bottom-right']
    first_page = camelot.read_pdf(
        '../data/A14-RMS/a14-rms-2021.pdf'
        , flavor='stream'
        , table_areas=[','.join(start_tl + start_br)]
        , pages=first
        )

    df_donaut_rms = pd.concat([df_donaut_rms, fix_multiple_cols_name(first_page[0].df)])
    
    # parsing full pages where osc continue
    full = jParams[DOF_YEAR]['parse']['full']['pages']
    print(str(dt.now())[:19], f"- Parsing full pages ({full}) where OSC's data is present")
    full_tl = jParams[DOF_YEAR]['parse']['full']['top-left']
    full_br = jParams[DOF_YEAR]['parse']['full']['bottom-right']
    full_pages = camelot.read_pdf(
        '../data/A14-RMS/a14-rms-2021.pdf'
        , flavor='stream'
        , table_areas=[','.join(full_tl + full_br)]
        , pages=full
        )
    
    for n in range(len(full_pages)):
        aux = fix_multiple_cols_name(full_pages[n].df)
        df_donaut_rms = pd.concat([df_donaut_rms, aux])
    
    # parsing last page where osc end (truncated)
    last = jParams[DOF_YEAR]['parse']['end']['pages']
    print(str(dt.now())[:19], f"- Parsing last page ({last}) where OSC's data ends\n")
    end_tl = jParams[DOF_YEAR]['parse']['end']['top-left']
    end_br = jParams[DOF_YEAR]['parse']['end']['bottom-right']
    last_page = camelot.read_pdf(
        '../data/A14-RMS/a14-rms-2021.pdf'
        , flavor='stream'
        , table_areas=[','.join(end_tl + end_br)]
        , pages=last
        )

    df_donaut_rms = pd.concat([df_donaut_rms, fix_multiple_cols_name(last_page[0].df)])
    print(str(dt.now())[:19], '- Parse process completed')
    
    # some cleaning over here
    print(str(dt.now())[:19], '- Applying a some cleaning to the data')
    df_donaut_rms = df_donaut_rms.rename(columns={0: 'RFC', 1: 'razon_social'})
    # remove headers when osc changes their social objective in pdf
    df_donaut_rms = df_donaut_rms[~((df_donaut_rms.RFC == 'RFC') & (df_donaut_rms.razon_social == 'Denominación Social'))].reset_index(drop=True)
    df_donaut_rms = df_donaut_rms[df_donaut_rms.RFC.str.len().isin([0, 12])].reset_index(drop=True)
    
    df_donaut_rms['razon_social'] = df_donaut_rms.razon_social.str.replace('\n', ' ')
    
    print(str(dt.now())[:19], '- Fixing splitted OSC names')
    # check if razon social was splitted in two, row merge needed
    df_donaut_rms[['merge_needed']] = (
        1*(df_donaut_rms.razon_social.shift(-1).str.len() == 0) 
        + 1*(df_donaut_rms.razon_social.shift(1).str.len() == 0)
    )
    df_donaut_rms[['merge_needed']] = (df_donaut_rms.merge_needed.shift(-1) + df_donaut_rms.merge_needed.shift(1))
    for i in df_donaut_rms[df_donaut_rms.merge_needed == 2].index:
        df_donaut_rms.loc[i, 'razon_social'] = df_donaut_rms.loc[i-1, 'razon_social'] + ' ' + df_donaut_rms.loc[i+1, 'razon_social']
    
    print(str(dt.now())[:19], '- Remove bad RFCs')
    df_donaut_rms = df_donaut_rms[df_donaut_rms.RFC.str.len() == 12]
    df_donaut_rms = df_donaut_rms[df_donaut_rms.RFC.str.isupper()]
    df_donaut_rms = df_donaut_rms[~df_donaut_rms.RFC.str.isalpha()].reset_index(drop=True)
    df_donaut_rms = df_donaut_rms.drop_duplicates("RFC").reset_index(drop=True)
    df_donaut_rms = df_donaut_rms.drop(columns=['merge_needed'])
    
    df_donaut_rms[['dof_year']] = DOF_YEAR
    
    print(str(dt.now())[:19], f'- Total unique OSCs retrived in {year}:', str(df_donaut_rms.shape[0]), '\n')
    
    df_osc_donaut_dof = pd.concat([df_osc_donaut_dof, df_donaut_rms])
    

In [ ]:
# accuracies = []
# whitespaces = []

# accuracies += [full_pages[n].parsing_report['accuracy']
# whitespaces += [full_pages[n].parsing_report['accuracy']]

# print('Average parsed accuracy', np.mean(accuracies).round(2))
# print('Average parsed whitespace', np.mean(whitespaces).round(2))

In [ ]:
df_osc_donaut_dof = df_osc_donaut_dof.rename(columns={'razon_social': 'sat_razon_social', 'RFC': 'rfc'})

In [ ]:
df_osc_donaut_dof.head()

## 4. Consolidate Datasets

In [ ]:
df_osc_cluni.shape

In [ ]:
df_osc_donaut_sat.shape

In [ ]:
df_osc = (
    df_osc_cluni
    .merge(df_osc_donaut_sat, how='outer', on='rfc')
    .merge(df_osc_donaut_dof, how='outer', on='rfc')
)

In [ ]:
df_osc = df_osc.drop_duplicates().reset_index(drop=True)

In [ ]:
df_osc.sample(5)

In [ ]:
df_osc.shape

In [ ]:
(~df_osc[['rfc', 'cluni', 'cluni_razon_social', 'sat_razon_social']].isna()).sum(axis=0)

In [ ]:
print('OSCs with RFC:', df_osc.rfc.nunique())
print('OSCs with CLUNI:', df_osc.cluni.nunique())

In [ ]:
with_cluni = ~df_osc.cluni.isna()
with_donaut = ~(df_osc.sat_year.isna() & df_osc.dof_year.isna())

In [ ]:
df_osc_cluni_not_donaut = df_osc[with_cluni & ~with_donaut].reset_index(drop=True)
df_osc_cluni_donaut = df_osc[with_cluni & with_donaut].reset_index(drop=True)
df_osc_donaut_not_cluni = df_osc[~with_cluni & with_donaut].reset_index(drop=True)

In [ ]:
print('OSCs CLUNI not Donaut:', df_osc_cluni_not_donaut.rfc.nunique())
print('OSCs CLUNI and Donaut:', df_osc_cluni_donaut.rfc.nunique())
print('OSCs Donaut not CLUNI :', df_osc_donaut_not_cluni.rfc.nunique())

In [ ]:
bad_rfc_cluni_not_donaut = df_osc_cluni_not_donaut.rfc.value_counts()[df_osc_cluni_not_donaut.rfc.value_counts() > 1].index.tolist()
bad_rfc_cluni_donaut = df_osc_cluni_donaut.rfc.value_counts()[df_osc_cluni_donaut.rfc.value_counts() > 1].index.tolist()
bad_rfc_donaut_not_cluni = df_osc_donaut_not_cluni.rfc.value_counts()[df_osc_donaut_not_cluni.rfc.value_counts() > 1].index.tolist()

In [ ]:
one_rfc_many_cluni = bad_rfc_cluni_not_donaut + bad_rfc_cluni_donaut + bad_rfc_donaut_not_cluni

In [ ]:
(
    df_osc[df_osc.rfc.isin(one_rfc_many_cluni)]
    .groupby(['rfc', 'estatus'])
    [['rfc']].count()
    .rename(columns={'rfc': 'count'})
    .reset_index(drop=False)
    .pivot('rfc', 'estatus', 'count')
    .sort_values('activa')
)